In [5]:
from aerobot.dataset import dataset_load
from aerobot.plot import *
# from aerobot.utils import print_taxonomy_info
from aerobot.io import load_results_dict, RESULTS_PATH, DATA_PATH
import numpy as np 
import os
from matplotlib.gridspec import GridSpec
import pandas as pd 
import subprocess
from typing import Dict

%load_ext autoreload
%autoreload 2

plot_configure_mpl()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# "Minimal list" of feature types.
FEATURE_TYPES = ['KO', 'embedding.geneset.oxygen', 'embedding.genome', 'chemical', 'aa_1mer', 'aa_3mer']

In [7]:
# Load in the phylogenetic cross-validation results for the specified feature types. 
phylo_cv_results = {feature_type:load_results_dict(os.path.join(RESULTS_PATH, f'phylo_cv_nonlinear_{feature_type}_ternary.json')) for feature_type in FEATURE_TYPES}

FileNotFoundError: [Errno 2] No such file or directory: '/home/prichter/Documents/aerobot/aerobot/../results/phylo_cv_nonlinear_aa_1mer_ternary.json'

# SI Figure 4

In [ ]:
def plot_si_figure_4a(taxonomy_coverage_df:pd.DataFrame, fig:plt.Figure=None, grid:GridSpec=None):

    # Make the easiest plot, which just shows percent coverage by phylogenetic level. 
    ax = fig.add_subplot(grid)
    n = len(taxonomy_coverage_df) # Number of habitats. 
    
    human_habitat_color, color = 'tab:blue', 'tab:gray'
    colors = [color if not is_human else human_habitat_color for is_human in taxonomy_coverage_df.habitat.str.contains('human')]
    # facecolors = ['none' if not is_human else 'tab:blue' for is_human in human_samples]
    sizes = [3 if not is_human else 20 for is_human in taxonomy_coverage_df.habitat.str.contains('human')]

    for i, level in enumerate(LEVELS):
        ax.scatter(np.array([i] * n), taxonomy_coverage_df[level].values, linewidth=0.5, facecolors=colors, s=sizes)# , edgecolors=edgecolors)

    x_vals, y_vals = [], []
    for i, level in enumerate(LEVELS): # Make sure these are plotted on top.
        x_vals.append(i)
        y_vals.append(taxonomy_coverage_df[level].mean())

    ax.plot(x_vals, y_vals, c='black', lw=1)
    ax.scatter(x_vals, y_vals, c='black', s=20)

    # Custom legend. Colors indicate annotation-free or annotation-full, and hatching indicates training or validation set. 
    handles = [plt.Rectangle((0, 0), 1, 1, facecolor=human_sample_color, edgecolor='black', linewidth=0.5)]
    handles.append(plt.Rectangle((0, 0), 1, 1, facecolor=color, edgecolor='black', linewidth=0.5))
    handles.append(plt.Rectangle((0, 0), 1, 1, facecolor='black', edgecolor='black', linewidth=0.5))

    labels = ['human habitats', 'non-human habitats', 'mean']
    ax.legend(handles, labels, fontsize=7, columnspacing=0.3, handletextpad=0.3, loc='lower left')

    ax.set_xticks(np.arange(len(LEVELS)), labels=LEVELS)
    ax.set_ylabel('percent coverage')
    ax.set_xlabel('phylogenetic level')
    ax.set_title('Phylogenetic label coverage in the Earth Microbiome Project')

In [ ]:
def plot_si_figure_4b(phylo_cv_results:Dict, fig:plt.Figure=None, grid:GridSpec=None):
    '''Phylogenetic cross-validation results for the minimal list of feature types.'''
    ax = fig.add_subplot(grid)
    
    for feature_type, results in phylo_cv_results.items():
        plot_phylo_cv(results, ax)
    

NameError: name 'Dict' is not defined

In [ ]:
fig = plt.figure(figsize=(10, 5))
grid = GridSpec(1, 1, width_ratios=(3, 1), wspace=0.2, hspace=0.3)

plot_si_figure_4a(earth_microbiome_df, grid=grid[0, 0], fig=fig)

plt.tight_layout()
plt.show()

In [ ]:
load_results_dict(os.path.join(RESULTS_PATH, 'phylo_cv_nonlinear_KO_ternary.json'))

{'scores': {'Phylum': [0.5655778860149446,
   0.440836712586159,
   0.6025999403658978,
   0.5689065517743881,
   0.5434111711483974,
   0.6310010440445223,
   0.5350620873285074,
   0.6033046926635822,
   0.6551037918887231,
   0.8384208384208384,
   0.5741948573192784,
   0.6403084301433436,
   0.48145019795993993,
   0.4878563931800002,
   0.4366290369163996,
   0.5430590049636547,
   0.5986750096032821,
   0.5169232171693494,
   0.6040940204055275,
   0.6087583200482521,
   0.6104197628434282,
   0.5310344827586208,
   0.45285832642916324,
   0.4059832238295446,
   0.5104032935713853],
  'Class': [0.5995321205405238,
   0.570546817453096,
   0.6305013171941863,
   0.36446030183727035,
   0.6226448466829307,
   0.6386465767909068,
   0.7380952380952381,
   0.7852758780110638,
   0.46480319367643314,
   0.6362869928014008,
   0.3948632136725321,
   0.5966550973186334,
   0.48891562443224296,
   0.4814232217305503,
   0.6375356350966107,
   0.5518931501705303,
   0.5855515224843718,
 